<a href="https://colab.research.google.com/github/Izuho/senior-thesis/blob/main/qiskit2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit
!pip install platexenc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 5.2 MB/s 
     |████████████████████████████████| 19.2 MB 1.2 MB/s 
     |████████████████████████████████| 240 kB 53.9 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 106 kB 8.7 MB/s 
     |████████████████████████████████| 943 kB 60.9 MB/s 
     |████████████████████████████████| 37.5 MB 30 kB/s 
     |████████████████████████████████| 49 kB 3.4 MB/s 
     |████████████████████████████████| 50 kB 5.1 MB/s 
     |████████████████████████████████| 4.1 MB 44.3 MB/s 
     |████████████████████████████████| 1.9 MB 58.4 MB/s 
     |████████████████████████████████| 112 kB 57.2 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.2-py3-none-any.whl size=12224 sha256=788f336f65fc2987a187bb672a32627c1d30b9060baf457fca240979136fdf9e
  Stored in directory: /root/.cache/pip/wheels/30/dc/90/0ee55cc

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [62]:
import numpy as np
X1 = np.load('drive/MyDrive/fmnist_train_feat.npy')[0:10]
y1 = np.load('drive/MyDrive/fmnist_train_Label.npy')[0:10]
X2 = np.load('drive/MyDrive/fmnist_test_feat.npy')[0:50]
y2 = np.load('drive/MyDrive/fmnist_test_Label.npy')[0:50]
print(X1.shape, y1.shape)

(10, 64) (10,)


In [64]:
from qiskit.circuit import Parameter, QuantumCircuit

def fraxis_gate(nx, ny, nz=None):
    nx, ny, nz = _validate(nx, ny, nz)
    theta = None
    if nz != 0:
      theta = 2*np.arctan(np.sqrt(nx**2+ny**2)/nz)
    else:
      theta = -np.pi
    phi = None
    if nx*ny>0:
      phi=np.arctan(ny/nx)
    elif nx*ny<0:
      phi=np.arctan(ny/nx)+np.pi
    else:
      phi=np.pi/2
    circ = QuantumCircuit(1)
    
    circ.rz(np.pi-phi,[0])
    circ.sx([0])
    circ.rz(theta+np.pi, [0])
    circ.sx([0])
    circ.rz(phi+np.pi,[0])
    
    return circ

def _validate(nx, ny, nz=None):
    if nz == None:
        nx = nx.real
        ny = ny.real
        nz2 = 1-nx**2.-ny**2.
        nz = np.sqrt(nz2).real if nz2>0 else 0
    return nx, ny, nz

In [65]:
def FraxisFeatureMap(num_qubits, data):
    circ = QuantumCircuit(num_qubits)
    for i in range(
        data.shape[0]//2
    ):
        circ.compose(fraxis_gate(data[2*i], data[2*i+1]), qubits=[i%num_qubits], inplace=True)
        if (i+1)%num_qubits == 0:
            for j in range(0,num_qubits,2):
                if j+1 < num_qubits:
                    circ.cz(j,j+1)
            for j in range(1,num_qubits,2):
                if j+1 < num_qubits:
                    circ.cz(j,j+1)        
    return circ

In [66]:
def FraxisAnsatz(num_qubits, params):
    circ = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        circ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    for j in range(0,num_qubits,2):
        if j+1 < num_qubits:
            circ.cz(j,j+1)
    for j in range(1,num_qubits,2):
        if j+1 < num_qubits:
            circ.cz(j,j+1)        
    return circ

In [67]:
def replace_FraxisAnsatz(num_qubits, target, params):
    circX = QuantumCircuit(num_qubits)
    circY = QuantumCircuit(num_qubits)
    circZ = QuantumCircuit(num_qubits)
    circXY = QuantumCircuit(num_qubits)
    circXZ = QuantumCircuit(num_qubits)
    circYZ = QuantumCircuit(num_qubits)
    for i in range(target):
        circX.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circYZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    circX.x(target)
    circY.y(target)
    circZ.z(target)
    circXY.u(np.pi, np.pi*0.25, np.pi*0.75, target)
    circXZ.u(np.pi*0.5, 0, np.pi, target)
    circYZ.u(np.pi*0.5, np.pi*0.5, np.pi*0.5, target)
    for i in range(target+1, num_qubits, 1):
        circX.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXY.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circXZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
        circYZ.compose(fraxis_gate(params[i,0], params[i,1]), qubits=[i], inplace=True)
    for j in range(0,num_qubits,2):
        if j+1 < num_qubits:
            circX.cz(j,j+1)
            circY.cz(j,j+1)
            circZ.cz(j,j+1)
            circXY.cz(j,j+1)
            circXZ.cz(j,j+1)
            circYZ.cz(j,j+1)
    for j in range(1,num_qubits,2):
        if j+1 < num_qubits:
            circX.cz(j,j+1)
            circY.cz(j,j+1)
            circZ.cz(j,j+1)
            circXY.cz(j,j+1)
            circXZ.cz(j,j+1)
            circYZ.cz(j,j+1)
    return [circX, circY, circZ, circXY, circXZ, circYZ]

In [68]:
from qiskit.primitives import Sampler
from qiskit.circuit import ClassicalRegister

class FraxClassify_sim():
    def __init__(self, n_qubits, layer_size, world_size, service):
        self.n_qubits = n_qubits
        self.layer_size = layer_size
        self.params = np.random.rand(layer_size, n_qubits, 2)*2/3
        #np.zeros((layer_size, n_qubits, 2)) + 1/np.sqrt(3)
        self.world_size =world_size
        self.service = service
        self.score = None
    def eval(self, X, y, sampler):
        acc, score= 0,0
        for a in range(X.shape[0]//self.world_size):
            qc = QuantumCircuit(self.n_qubits*self.world_size)
            feature_map = []
            for b in range(self.world_size):
                feature_map.append(FraxisFeatureMap(self.n_qubits, X[a*self.world_size+b]))
            for b in range(self.layer_size):
                for c in range(self.world_size):
                    qc.compose(
                        feature_map[c], 
                        qubits=range(self.n_qubits*c,self.n_qubits*(c+1),1), 
                        inplace=True)
                    qc.compose(
                        FraxisAnsatz(self.n_qubits, self.params[b]), 
                        qubits=range(self.n_qubits*c,self.n_qubits*(c+1),1), 
                        inplace=True
                    )
            
            cr = ClassicalRegister(self.world_size)
            qc.add_register(cr)
            qc.measure(
                range(0,self.world_size*self.n_qubits,self.n_qubits),
                cr
            )
                    
            result = sampler.run(
                circuits=[qc],
            ).result().quasi_dists
            Zexp = np.zeros(self.world_size)
            for bits in result[0]:
                for e in range(self.world_size):
                    if (bits>>(e))%2 == 1:
                        Zexp[e] -= result[0][bits]
                    else:
                        Zexp[e] += result[0][bits]
                        
            acc += np.sum(np.where(y[self.world_size*a:self.world_size*(a+1)]*Zexp>0,1,0))
            score += np.sum(y[self.world_size*a:self.world_size*(a+1)]*Zexp)
        print(acc, score*2)
        return acc, score*2
        
    def fit_and_eval(self, X, y, X2, y2):
        sampler = Sampler()
        for a in range(self.layer_size):
            for b in range(self.n_qubits):
                R = np.zeros((3,3))
                for c in range(y.shape[0]//self.world_size):
                    feature_map = []
                    for d in range(self.world_size):
                        feature_map.append(FraxisFeatureMap(self.n_qubits, X[c*self.world_size+d]))
                    qcs = []
                    for d in range(6):
                        qcs.append(QuantumCircuit(self.n_qubits*self.world_size))
                    for d in range(a):
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    feature_map[f], 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )
                            
                        original_ansatz = FraxisAnsatz(self.n_qubits, self.params[d])
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    original_ansatz, 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )

                    for d in range(6):
                        for f in range(self.world_size):
                            qcs[d].compose(
                                feature_map[f], 
                                qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                inplace=True
                            )
                        
                    ansatzs = replace_FraxisAnsatz(self.n_qubits, b, self.params[a])
                    for d in range(6):
                        for f in range(self.world_size):
                            qcs[d].compose(
                                ansatzs[d], 
                                qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                inplace=True
                            )
                                        
                    for d in range(a+1,self.layer_size,1):
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    feature_map[f], 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )
                            
                        original_ansatz = FraxisAnsatz(self.n_qubits, self.params[d])
                        for e in range(6):
                            for f in range(self.world_size):
                                qcs[e].compose(
                                    original_ansatz, 
                                    qubits=range(self.n_qubits*f,self.n_qubits*(f+1),1), 
                                    inplace=True
                                )

                    for d in range(6):
                        cr = ClassicalRegister(self.world_size)
                        qcs[d].add_register(cr)
                        qcs[d].measure(
                            range(0,self.world_size*self.n_qubits,self.n_qubits),
                            cr
                        )
                    result = sampler.run(
                        circuits=qcs,
                    ).result().quasi_dists

                    r6s = np.zeros((6,self.world_size))
                    for d in range(6):
                        #print(result[d])
                        for bits in result[d]:
                            for e in range(self.world_size):
                                if (bits>>(e))%2 == 1:
                                    r6s[d, e] -= result[d][bits]
                                else:
                                    r6s[d, e] += result[d][bits]
                                    
                    R[0,0] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * 2 * r6s[0])
                    R[0,1] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * (2 * r6s[3] - r6s[0] - r6s[1]))
                    R[0,2] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * (2 * r6s[4] - r6s[0] - r6s[2]))
                    R[1,1] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * 2 * r6s[1])
                    R[1,2] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * (2 * r6s[5] - r6s[1] - r6s[2]))
                    R[2,2] += np.sum(y[c*self.world_size:(c+1)*self.world_size] * 2 * r6s[2])
                R[1,0] = R[0,1]
                R[2,0] = R[0,2]
                R[2,1] = R[1,2]

                eigenvalues, eigenvectors = np.linalg.eigh(R)
                self.params[a,b,:] = eigenvectors[0:2,np.argmax(eigenvalues)]
                print(np.max(eigenvalues))
                self.eval(X,y,sampler)

In [69]:
model = FraxClassify_sim(3, 2, 1, None)
for i in range(3):
    model.fit_and_eval(X1, y1, X2, y2)
    print('_______________NEW________________')

4.704876004872279
7 3.651592493707802
6.440961171795754
7 3.6431867592163294
6.590030721360261
7 6.590030721360251
7.411003898278733
7 7.411003898278734
7.411003898278738
7 7.411003898278733
7.411003898278733
7 7.411003898278733
_______________NEW________________
7.518293823971747
7 7.518293823971737
7.620500589400846
7 7.6205005894008515
7.703478576470457
7 7.70347857647046
7.760302638017869
7 7.760302638017873
7.760302638017882
7 7.7603026380178735
7.760302638017875
7 7.760302638017874
_______________NEW________________
7.854011660293231
7 7.803928700206639
7.882222435201325
7 7.882222435201328
7.893336452091178
7 7.893336452091184
7.899342908035012
7 7.899342908035004
7.899342908035014
7 7.8993429080350035
7.899342908035011
7 7.899342908035008
_______________NEW________________


In [73]:
import qiskit.quantum_info as qi
nxs=[-0.01,0,0.01]
nys=[-0.01,0,0.01]
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1j],[1j,0]])
Z=np.array([[1,0],[0,-1]])
np.set_printoptions(precision=3,suppress=True)
for nx in nxs:
  for ny in nys:
    circ=fraxis_gate(nx,ny)
    op = qi.Operator(circ)
    print(nx,ny)
    nx,ny,nz=_validate(nx,ny)
    print(nx*X+ny*Y+nz*Z)
    print(op.data,'\n')

-0.01 -0.01
[[ 1.  +0.j   -0.01+0.01j]
 [-0.01-0.01j -1.  +0.j  ]]
[[-1.  -0.j   -0.01+0.01j]
 [-0.01-0.01j  1.  -0.j  ]] 

-0.01 0
[[ 1.  +0.j -0.01+0.j]
 [-0.01+0.j -1.  +0.j]]
[[-1.-0.j   -0.+0.01j]
 [-0.-0.01j  1.-0.j  ]] 

-0.01 0.01
[[ 1.  +0.j   -0.01-0.01j]
 [-0.01+0.01j -1.  +0.j  ]]
[[-1.  -0.j    0.01+0.01j]
 [ 0.01-0.01j  1.  -0.j  ]] 

0 -0.01
[[ 1.+0.j    0.+0.01j]
 [ 0.-0.01j -1.+0.j  ]]
[[-1.-0.j   -0.+0.01j]
 [-0.-0.01j  1.-0.j  ]] 

0 0
[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]
[[-1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j]] 

0 0.01
[[ 1.+0.j    0.-0.01j]
 [ 0.+0.01j -1.+0.j  ]]
[[-1.-0.j   -0.+0.01j]
 [-0.-0.01j  1.-0.j  ]] 

0.01 -0.01
[[ 1.  +0.j    0.01+0.01j]
 [ 0.01-0.01j -1.  +0.j  ]]
[[-1.  -0.j    0.01+0.01j]
 [ 0.01-0.01j  1.  -0.j  ]] 

0.01 0
[[ 1.  +0.j  0.01+0.j]
 [ 0.01+0.j -1.  +0.j]]
[[-1.-0.j   -0.+0.01j]
 [-0.-0.01j  1.-0.j  ]] 

0.01 0.01
[[ 1.  +0.j    0.01-0.01j]
 [ 0.01+0.01j -1.  +0.j  ]]
[[-1.  -0.j   -0.01+0.01j]
 [-0.01-0.01j  1.  -0.j  ]] 

